In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from xgboost import XGBClassifier, XGBRegressor
import xgboost as xgb
from sklearn.model_selection import train_test_split
import os
import sys
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import accuracy_score, roc_auc_score
from scipy.optimize import minimize
SEED = 42
def feval(pred_proba, dtrain):
    y_true = dtrain.get_label().astype(int)
    roc = roc_auc_score(y_true, pred_proba)
    return "roc", roc

In [2]:
test = pd.read_csv("../input/jpeg-melanoma-256x256/test.csv")
train = pd.read_csv("../input/jpeg-melanoma-256x256/train.csv")
print(test.columns)
print(train.columns)


Index(['image_name', 'patient_id', 'sex', 'age_approx',
       'anatom_site_general_challenge', 'width', 'height'],
      dtype='object')
Index(['image_name', 'patient_id', 'sex', 'age_approx',
       'anatom_site_general_challenge', 'diagnosis', 'benign_malignant',
       'target', 'tfrecord', 'width', 'height'],
      dtype='object')


In [3]:
oof_list = []
sub_list = []
futter = ["", "_TTA1"]
No_list = [0, 1, 2]
fig_size_list = [256, 384, 512, 768]
for fut in futter:
    for fig_size in fig_size_list:
        for No in No_list:
            dir_path = f"../exp/{fig_size}-{No}"
            oof_path = os.path.join(dir_path, f"oof_No{No}_{fig_size}{fut}.csv")
            if os.path.isfile(oof_path):
                oof_list.append(oof_path)
            sub_path = os.path.join(dir_path, f"submission_No{No}_{fig_size}{fut}.csv")
            if os.path.isfile(sub_path):
                sub_list.append(sub_path)
print(len(oof_list), len(sub_list))

18 18


In [4]:
oof_cols = [f"oof_{i}" for i in range(len(oof_list))]
oof_df = pd.DataFrame(np.zeros((len(train), len(oof_list))), columns=oof_cols)
pred_cols = [f"pred_{i}" for i in range(len(oof_list))]
pred_df = pd.DataFrame(np.zeros((len(test), len(sub_list))), columns=pred_cols)
for i in range(len(oof_list)):
    oof_df.iloc[:, i] = pd.read_csv(oof_list[i]).reset_index()["0"]
    pred_df.iloc[:, i] = pd.read_csv(sub_list[i]).reset_index()["target"]
    roc = roc_auc_score(train["target"].values, oof_df.iloc[:, i].values)
    print(f"{oof_list[i].split('/')[-1]}, roc:{roc:.5f}")
roc = roc_auc_score(train["target"].values, oof_df.mean(axis=1).values)
print(f"avg, roc:{roc:.5f}")
print(oof_df.shape, pred_df.shape)
sub = pd.read_csv(f"../input/jpeg-melanoma-{fig_size}x{fig_size}/sample_submission.csv")
sub["target"] = pred_df.mean(axis=1).values
sub.to_csv(f"../exp/ensemble/submission_avg.csv", index=False)
sub

oof_No0_256.csv, roc:0.87891
oof_No1_256.csv, roc:0.87277
oof_No2_256.csv, roc:0.89128
oof_No0_384.csv, roc:0.89989
oof_No1_384.csv, roc:0.87027
oof_No1_512.csv, roc:0.87592
oof_No2_512.csv, roc:0.88975
oof_No0_768.csv, roc:0.84800
oof_No2_768.csv, roc:0.86828
oof_No0_256_TTA1.csv, roc:0.87891
oof_No1_256_TTA1.csv, roc:0.87277
oof_No2_256_TTA1.csv, roc:0.89128
oof_No0_384_TTA1.csv, roc:0.89989
oof_No1_384_TTA1.csv, roc:0.87027
oof_No1_512_TTA1.csv, roc:0.87592
oof_No2_512_TTA1.csv, roc:0.88975
oof_No0_768_TTA1.csv, roc:0.84800
oof_No2_768_TTA1.csv, roc:0.86828
avg, roc:0.90530
(33126, 18) (10982, 18)


,image_name,target
0,ISIC_0052060,0.001143
1,ISIC_0052349,0.000433
2,ISIC_0058510,0.000071
3,ISIC_0073313,0.000094
4,ISIC_0073502,0.011223
...,...,...
10977,ISIC_9992485,0.002102
10978,ISIC_9996992,0.008063
10979,ISIC_9997917,0.035766
10980,ISIC_9998234,0.000747


In [35]:
sub0 = pd.read_csv("../exp/ensemble/submission_avg.csv")
a = sub["target"] - sub0["target"]
a.sum()

-2.0774668877537872e-16

In [25]:
Y_values = train["target"].values
predictions = []
lls = []
wghts = []
# Define the objective function to optimize MSE by using ScyiPy algorithm.
def roc_func(weights):
    ''' scipy minimize will pass the weights as a numpy array '''
    final_prediction = 0
    for weight, prediction in zip(weights, predictions):
        final_prediction += weight*prediction

    return 1.0 - roc_auc_score(Y_values, final_prediction)



In [40]:
lls

[]

In [41]:
# Optimization runs 100 times.
_weights = []

skf = StratifiedKFold(n_splits=5, random_state=SEED, shuffle=True)
for i, (train_idx, val_idx) in enumerate(skf.split(oof_df.values, train["target"].values)):
    X_train = oof_df.loc[train_idx].values
    X_val = oof_df.loc[val_idx].values
    y_train = train.loc[train_idx, "target"].values
    y_val = train.loc[val_idx, "target"].values
    predictions = []
    lls = []
    wghts = []
    for j in range(oof_df.shape[1]):
        predictions.append(np.array(oof_df.iloc[train_idx, j]))
    Y_values = train.loc[train_idx, "target"].values

    def roc_func(weights):
        ''' scipy minimize will pass the weights as a numpy array '''
        final_prediction = 0
        for weight, prediction in zip(weights, predictions):
            final_prediction += weight*prediction
        return 1.0 - roc_auc_score(Y_values, final_prediction)
    
    for i in range(100):
        starting_values = np.random.uniform(size=oof_df.shape[1])
        # cons are given as constraints.
        cons = ({'type':'eq','fun':lambda w: 1-sum(w)})
        bounds = [(0,1)]*oof_df.shape[1]
        
        res = minimize(roc_func, starting_values, constraints=cons,
                    bounds = bounds, method='SLSQP')
        lls.append(res['fun'])
        wghts.append(res['x'])
    bestSC = np.min(lls)
    bestWght = wghts[np.argmin(lls)]
    _weights.append(bestWght)
    print('Best Weights: {weights:}'.format(weights=bestWght))
    print('Train Ensemble ROC: {:.7f}'.format(roc_auc_score(y_train, np.sum(bestWght * X_train, axis=1))))
    print('Val   Ensemble ROC: {:.7f}'.format(roc_auc_score(y_val, np.sum(bestWght * X_val, axis=1))))

    

Best Weights: [1.45772178e-01 0.00000000e+00 4.01546397e-01 3.13553659e-01
 1.11022302e-16 1.39127766e-01 5.55111512e-17 0.00000000e+00
 2.77555756e-17]
Train Ensemble ROC: 0.9086885
Val Ensemble ROC: 0.9157308
Best Weights: [2.76606049e-01 1.02954419e-01 3.17384842e-01 1.73673162e-01
 5.55111512e-17 7.88738386e-02 1.66533454e-16 5.05076894e-02
 0.00000000e+00]
Train Ensemble ROC: 0.9120338
Val Ensemble ROC: 0.9077630
Best Weights: [3.13638771e-01 3.05531393e-01 1.89386333e-01 9.61827937e-02
 1.11022302e-16 0.00000000e+00 9.52607089e-02 0.00000000e+00
 0.00000000e+00]
Train Ensemble ROC: 0.9132113
Val Ensemble ROC: 0.9049703
Best Weights: [4.06614112e-01 3.11035099e-01 1.63183096e-01 9.44152945e-02
 1.11022302e-16 1.11022302e-16 2.47523991e-02 5.55111512e-17
 1.11022302e-16]
Train Ensemble ROC: 0.9086217
Val Ensemble ROC: 0.9175584
Best Weights: [0.2731937  0.         0.35442008 0.34555852 0.         0.
 0.0268277  0.         0.        ]
Train Ensemble ROC: 0.9141144
Val Ensemble ROC: 

In [49]:
pred = np.sum(np.array(_weights).mean(axis=0)*oof_df.values, axis=1)
roc = roc_auc_score(train["target"].values, pred)
print("ROC", roc)
y_test = np.sum(np.array(_weights).mean(axis=0)*pred_df.values, axis=1)
sub["target"] = y_test
sub.to_csv(f"../exp/ensemble/submission_blend.csv", index=False)
sub

ROC 0.9132924795606606


,image_name,target
0,ISIC_0052060,0.000342
1,ISIC_0052349,0.000034
2,ISIC_0058510,0.000016
3,ISIC_0073313,0.000020
4,ISIC_0073502,0.002427
...,...,...
10977,ISIC_9992485,0.000779
10978,ISIC_9996992,0.005674
10979,ISIC_9997917,0.016131
10980,ISIC_9998234,0.000163


In [5]:
# 分類問題として0or1分類をする
# X_train, X_val, y_train, y_val = train_test_split(oof_df.values, train["target"].values, test_size=0.2, random_state=SEED)
# reg_xgb = GridSearchCV(xgb_model,
#                    {'max_depth': [2,4,6],
#                     "scale_pos_weight ": [10, 50, 100],
#                     "reg_lambda" : [1e-2, 1.0, 1e3],
#                     'n_estimators': [50,100,200]},
#                     verbose=1,
#                     n_jobs=16)
# param_dist = {
#     'objective':'binary:logistic',
#     'n_estimators':64,
#     'max_depth': 8,
#     'learning_rate': 1e-3,
#     'reg_lambda': 1e-1,
#     'scale_pos_weight': 100.0,
#     'random_state': SEED,
#     'num_parallel_tree': 16,   
# }


# clf = xgb.XGBRFClassifier(**param_dist)
# clf.fit(
#     X_train, 
#     y_train,
#     eval_set=[(X_train, y_train), (X_val, y_val)],
#     eval_metric='logloss',
#     verbose=True,
#     early_stopping_rounds=100
#     )

# evals_result = clf.evals_result()
# importance = clf.feature_importances_
# y_test = clf.predict_proba(pred_df.values)

[0]	validation_0-logloss:0.69246	validation_1-logloss:0.69247
Multiple eval metrics have been passed: 'validation_1-logloss' will be used for early stopping.

Will train until validation_1-logloss hasn't improved in 100 rounds.


In [21]:
# 分類問題として0or1分類をする
X_train, X_val, y_train, y_val = train_test_split(oof_df.values, train["target"].values, test_size=0.2, random_state=SEED)
param_dist = {
    'objective':'reg:squarederror',
    # 'n_estimators':64,
    'max_depth': 8,
    'learning_rate': 1e-1,
    'reg_lambda': 1e-1,
    'scale_pos_weight': 100.0,
    'random_state': SEED,
    'num_parallel_tree': 16,
    'eta': 0.1,
    'max_delta_step': 2.0
}

dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)
evals = [(dtrain, 'train'), (dval, 'val')]
evals_result = {}
bst = xgb.train(param_dist,
                dtrain,
                num_boost_round=1000,
                early_stopping_rounds=50,
                evals=evals,
                evals_result=evals_result,
                feval=feval,
                )

importance = bst.feature_importances_
y_pred_proba = bst.predict(dtest)

c:0.69453
[286]	train-rmse:0.08458	val-rmse:0.19259	train-roc:0.99994	val-roc:0.69461
[287]	train-rmse:0.08432	val-rmse:0.19253	train-roc:0.99994	val-roc:0.69450
[288]	train-rmse:0.08411	val-rmse:0.19246	train-roc:0.99994	val-roc:0.69449
[289]	train-rmse:0.08378	val-rmse:0.19235	train-roc:0.99994	val-roc:0.69314
[290]	train-rmse:0.08354	val-rmse:0.19230	train-roc:0.99994	val-roc:0.69334
[291]	train-rmse:0.08330	val-rmse:0.19225	train-roc:0.99994	val-roc:0.69717
[292]	train-rmse:0.08303	val-rmse:0.19219	train-roc:0.99994	val-roc:0.69780
[293]	train-rmse:0.08286	val-rmse:0.19219	train-roc:0.99994	val-roc:0.69772
[294]	train-rmse:0.08262	val-rmse:0.19207	train-roc:0.99994	val-roc:0.69775
[295]	train-rmse:0.08241	val-rmse:0.19195	train-roc:0.99994	val-roc:0.69778
[296]	train-rmse:0.08235	val-rmse:0.19190	train-roc:0.99994	val-roc:0.69804
[297]	train-rmse:0.08197	val-rmse:0.19189	train-roc:0.99994	val-roc:0.69365
[298]	train-rmse:0.08189	val-rmse:0.19186	train-roc:0.99994	val-roc:0.69368
[2

AttributeError: 'Booster' object has no attribute 'feature_importances_'

In [ ]:
y_test = clf.predict(pred_df.values)
print(y_test.shape)
sub = pd.read_csv(f"../input/jpeg-melanoma-{fig_size}x{fig_size}/sample_submission.csv")
sub["target"] = y_test
sub.to_csv(f"../exp/ensemble/submission_xgblogit.csv", index=False)
sub
# kaggle competitions submit -c siim-isic-melanoma-classification -f exp/ensemble/submission_xgblogit.csv -m "Message"

In [ ]:
param_dist = {
    'objective':'reg:linear',
    # 'n_estimators':64,
    'max_depth': 8,
    'learning_rate': 1e-1,
    'reg_lambda': 1e-1,
    'scale_pos_weight': 100.0,
    'lambda	': 10.0,
    'random_state': SEED,
    'num_parallel_tree': 16,
    'eta': 0.1,
    'max_delta_step': 10.0
}
skf = StratifiedKFold(n_splits=5, random_state=SEED, shuffle=True)
importance_df = pd.DataFrame(np.zeros((5, len(oof_list))), columns=oof_cols)
for i, (train_idx, val_idx) in enumerate(skf.split(oof_df.values, train["target"].values)):
    X_train = oof_df.loc[train_idx].values
    X_val = oof_df.loc[val_idx].values
    y_train = train.loc[train_idx, "target"].values
    y_val = train.loc[val_idx, "target"].values

    dtrain = xgb.DMatrix(X_train, label=y_train)
    dval = xgb.DMatrix(X_val, label=y_val)
    evals = [(dtrain, 'train'), (dval, 'val')]
    evals_result = {}
    bst = xgb.train(param_dist,
                    dtrain,
                    num_boost_round=1000,
                    early_stopping_rounds=50,
                    evals=evals,
                    evals_result=evals_result,
                    feval=feval,
                    )

    importance_df.iloc[i] = bst.feature_importances_
    y_pred_proba = bst.predict(dtest)